In [34]:
import requests
from collections import Counter
from datetime import datetime

In [42]:
def openstates_v2_request (payload):
    """
    payload: string correspondong to a query.
    return: JSON object.
    """
    headers = {'x-api-key': API_KEY,
    'Content-Type': "application/x-www-form-urlencoded",}
    url = "https://openstates.org/graphql"
    response = requests.post(url, data=payload, headers=headers)
    return response.json()

def get_bill (identifier, state_name="California", session_year="20192020"):
    """
    Should return one bill given an identifier (e.g. 'AB 2'), a state, and a session.
    """
    payload = 'query= {bill(jurisdiction: "%s", session: "%s", identifier: "%s") {id}}' % (state_name, session_year, identifier)
    return openstates_v2_request(payload)

def all_bills (after="", jurisdiction="California", session="20192020"):
    """
    Loads the first 100 bills after the 'after' cursor. 
    Only returns (openstates) id, identifier (eg. 'AB 2'), title, classification, 
    updatedAt, createdAt, fromOrganization, and totalCount
    """
    payload = 'query= { bills(first: 100, jurisdiction: "%s", session: "%s", after: "%s") { totalCount pageInfo { startCursor endCursor hasNextPage } edges { bill:node { id identifier title classification updatedAt createdAt fromOrganization { id name} } } } } ' % (jurisdiction, session, after)
    return openstates_v2_request(payload)

In [3]:
STATES_DICT = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [4]:
SESSION_YEAR_DICT = {
        'AK': '31',
        'AL': '2019rs',
        'AR': '2019',
        'AZ': '54th-1st-regular',
        'CA': '20192020',
        'CO': '2019A',
        'CT': '2019',
        'DE': '150',
        'FL': '2019',
        'GA': '2019_20',
        'HI': '2019 Regular Session',
        'IA': '2019-2020',
        'ID': '2019',
        'IL': '101st',
        'IN': '2019',
        'KS': '2019-2020',
        'KY': '2019RS',
        'LA': '2019',
        'MA': '191st',
        'MD': '2019',
        'ME': '129',
        'MI': '2019-2020',
        'MN': '2019-2020',
        'MO': '2019',
        'MS': '2019',
        'MT': '2019',
        'NC': '2019',
        'ND': '66',
        'NE': '106',
        'NH': '2019',
        'NJ': '218',
        'NM': '2019',
        'NV': '80',
        'NY': '2019-2020',
        'OH': '133',
        'OK': '2019-2020',
        'OR': '2019 Regular Session',
        'PA': '2019-2020',
        'RI': '2019',
        'SC': '2019-2020',
        'SD': '2019',
        'TN': '111',
        'TX': '86',
        'UT': '2019',
        'VA': '2019',
        'VT': '2019-2020',
        'WA': '2019-2020',
        'WI': '2019',
        'WV': '2019',
        'WY': '2019'
}

In [26]:
def download_all_for_state (state):
    """
    Takes a state (full name, not abbreviation) and returns a list of all of its bills.
    """
    jurisdiction = state
    state_abbreviation = STATES_DICT[state]
    session = SESSION_YEAR_DICT[state_abbreviation]
    batch_100_bills = all_bills(after="", session = session, jurisdiction=jurisdiction)
    full_list_of_bills = []
    total_bills = batch_100_bills['data']['bills']['totalCount']
    for bill in batch_100_bills['data']['bills']['edges']:
        full_list_of_bills.append(bill)
    # Keep downloading bills until all are stored in a list.
    has_next = batch_100_bills['data']['bills']['pageInfo']['hasNextPage']
    while has_next:
        batch_100_bills = all_bills(after=batch_100_bills['data']['bills']['pageInfo']['endCursor'], 
                                    session=session, jurisdiction=jurisdiction)
        for bill in batch_100_bills['data']['bills']['edges']:
            full_list_of_bills.append(bill)
        has_next = batch_100_bills['data']['bills']['pageInfo']['hasNextPage']
    #check that we've downloaded the 'totalCount' of bills.
    print(total_bills == len(full_list_of_bills), total_bills, state)
    return full_list_of_bills

In [27]:
def download_all_states():
    """
    Returns a dictionary of states with a key of the state abbreviation and a value of
    a list of bills from the current session.
    """
    state_bill_dictionary = {}
    for state in STATES_DICT.keys():
        bill_list = download_all_for_state(state)
        state_bill_dictionary[state] = bill_list
    return state_bill_dictionary

In [28]:
state_bill_dict = download_all_states()

True 133 Alabama
True 0 Alaska
True 1385 Arizona
True 1419 Arkansas
True 3497 California
True 451 Colorado
True 3634 Connecticut
True 127 Delaware
True 3399 Florida
True 1635 Georgia
True 3970 Hawaii
True 481 Idaho
True 1182 Illinois
True 1461 Indiana
True 1786 Iowa
True 661 Kansas
True 1143 Kentucky
True 73 Louisiana
True 1253 Maine
True 2471 Maryland
True 11903 Massachusetts
True 580 Michigan
True 4720 Minnesota
True 3146 Mississippi
True 1976 Missouri
True 1057 Montana
True 790 Nebraska
True 517 Nevada
True 996 New Hampshire
True 9366 New Jersey
True 1613 New Mexico
True 11857 New York
True 529 North Carolina
True 983 North Dakota
True 309 Ohio
True 2838 Oklahoma
True 2620 Oregon
True 1223 Pennsylvania
True 1359 Rhode Island
True 1844 South Carolina
True 582 South Dakota
True 3665 Tennessee
True 8539 Texas
True 834 Utah
True 760 Vermont
True 3156 Virginia
True 2515 Washington
True 2119 West Virginia
True 193 Wisconsin
True 503 Wyoming


In [36]:
bill_title_dict = {}
for state in state_bill_dict:
    bill_title_dict[state] = [bill['bill']['identifier'] for bill in state_bill_dict[state]]

In [41]:
for state in bill_title_dict:
    count = Counter(bill_title_dict[state])
    duplicate_bills = [bill for bill in count if count[bill] > 1]
    print(state, len(duplicate_bills))

Alabama 0
Alaska 0
Arizona 0
Arkansas 0
California 752
Colorado 0
Connecticut 0
Delaware 0
Florida 0
Georgia 0
Hawaii 0
Idaho 0
Illinois 0
Indiana 0
Iowa 0
Kansas 0
Kentucky 0
Louisiana 0
Maine 0
Maryland 0
Massachusetts 0
Michigan 0
Minnesota 0
Mississippi 0
Missouri 0
Montana 0
Nebraska 0
Nevada 0
New Hampshire 0
New Jersey 0
New Mexico 0
New York 0
North Carolina 0
North Dakota 0
Ohio 0
Oklahoma 0
Oregon 0
Pennsylvania 0
Rhode Island 0
South Carolina 0
South Dakota 0
Tennessee 0
Texas 0
Utah 0
Vermont 0
Virginia 0
Washington 300
West Virginia 0
Wisconsin 0
Wyoming 0
